In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

sns.set(
    { "figure.figsize": (10, 8) },
    color_codes=True, font_scale=1.0)

In [2]:
energy = pd.read_csv("energydata_complete.csv")
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [4]:
data = energy[['T2', 'T6']]

# Split the data into input (X) and target (y) variables
X = data[['T2']]
y = data['T6']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict the target variable (T6) using the test input data (T2)
y_pred = lr.predict(X_test)

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
round(rmse,3)

3.633

### Linear

In [5]:
# Remove the specified columns
energy = energy.drop(["date", "lights"], axis=1)

# Set the target variable
y = energy["Appliances"]
X = energy.drop("Appliances", axis=1)

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the dataset using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit a multiple linear regression model
lr.fit(X_train_scaled, y_train)

# Predict the target variable for the training set
y_pred_train = lr.predict(X_train_scaled)

# Calculate the Mean Absolute Error (MAE) for the training set
mae_train = mean_absolute_error(y_train, y_pred_train)

round(mae_train, 3)

53.742

In [6]:
# Calculate the Mean Squared Error (MSE) for the training set
mse_train = mean_squared_error(y_train, y_pred_train)

# Calculate the Root Mean Squared Error (RMSE) for the training set
rmse_train = round(np.sqrt(mse_train),3)
rmse_train

95.216

In [7]:
# Predict the target variable for the test set
y_pred_test = lr.predict(X_test_scaled )

# Calculate the Mean Absolute Error (MAE) for the test set
mae_test = round(mean_absolute_error(y_test, y_pred_test),3)
mae_test

53.643

In [8]:
# Calculate the Mean Squared Error (MSE) for the test set
mse_test = mean_squared_error(y_test, y_pred_test)

# Calculate the Root Mean Squared Error (RMSE) for the test set
rmse_test = round(np.sqrt(mse_test),3)
rmse_test

93.64

### Ridge

In [9]:
from sklearn.linear_model import Ridge

rdg = Ridge()
rdg.fit(X_train_scaled, y_train)

# Predictions
y_test_pred = rdg.predict(X_test_scaled)

mse_test = mean_squared_error(y_test, y_test_pred, squared=False)

print("Root Mean Squared Error (RMSE) for test set:", round(np.sqrt(mse_test), 3))

Root Mean Squared Error (RMSE) for test set: 9.68


### Lasso

In [10]:
from sklearn.linear_model import Lasso

# Train the Lasso regression model with default parameters
lss = Lasso()
lss.fit(X_train_scaled, y_train)

# Obtain the feature weights (coefficients)
feature_weights = lss.coef_
print("Feature Weights:", feature_weights)

Feature Weights: [  0.          56.94451601   0.          -0.           0.
   0.          -0.           0.          -0.           0.
   0.          -0.          -0.          -0.           0.
 -26.10288797  -0.          -0.           0.          -0.
 -50.29397595   5.14314792   0.          -0.          -0.
  -0.        ]


In [11]:
y_test_pred = lss.predict(X_test_scaled)

# Calculate Mean Absolute Error (MAE) for training and test sets
mse_test = mean_squared_error(y_test, y_test_pred)
print("Root Mean Squared Error (RMSE) for test set:", round(np.sqrt(mse_test),3))

Root Mean Squared Error (RMSE) for test set: 99.424
